In [1]:
import os
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from dotenv import load_dotenv

import shutil
import stat

## Clone Github Repository

In [2]:
%pwd

'd:\\iNeuron\\GenAI\\GenerativeAI\\Source_code_analysis'

In [3]:
!mkdir test_repo

A subdirectory or file test_repo already exists.


In [4]:
def remove_readonly(func, path, _):
    """Clear the readonly bit and reattempt the removal."""
    os.chmod(path, stat.S_IWRITE)
    func(path)

# Define the repository URL and the path to clone to
repo_url = "https://github.com/entbappy/End-to-end-ML-Project-Implementation"
repo_path = "test_repo/"

# Check if the directory exists
if os.path.exists(repo_path):
    # If it exists, remove the directory with error handling
    shutil.rmtree(repo_path, onerror=remove_readonly)

# Clone the repository
Repo.clone_from(repo_url, to_path=repo_path)

print(f"Repository cloned to {repo_path}")


Repository cloned to test_repo/


In [5]:
repo_path = "test_repo/"

loader = GenericLoader.from_filesystem(repo_path+'/src/mlProject',
                                        glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [6]:
documents = loader.load()

In [7]:
# documents

### Chunkings

In [8]:
document_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON, 
                                                                 chunk_size=200, 
                                                                 chunk_overlap=200)

In [9]:
text = document_splitter.split_documents(documents=documents)
text[0].page_content

'import os\nimport sys\nimport logging\n\nlogging_str = "[%(asctime)s: %(levelname)s: %(module)s: %(message)s]"'

In [10]:
len(text)

169

### Embedding model

In [11]:
# Load environment variables from .env file
load_dotenv()

True

In [16]:
# Retrieve the API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# print(OPENAI_API_KEY)

In [13]:
embeddings = OpenAIEmbeddings(disallowed_special=())

c:\Users\sahil\.conda\envs\genai\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


### Knowledge base (vector DB)

In [14]:
vectordb = Chroma.from_documents(text, embedding=embeddings, persist_directory='./data')
vectordb.persist()

### LLM Wrapper

In [17]:
# llm = ChatOpenAI(model_name="gpt-4")
llm = ChatOpenAI()
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":3}), memory=memory)

c:\Users\sahil\.conda\envs\genai\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [18]:
question = "what is DataIngestion class?"
result = qa(question)
print(result['answer'])

c:\Users\sahil\.conda\envs\genai\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The purpose of the DataIngestion class is to handle data ingestion tasks by initializing with a DataIngestionConfig object.


In [ ]:
aa

In [ ]:
import os
import time
import random
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
import shutil
import stat
import openai  # Import the OpenAI library

def remove_readonly(func, path, _):
    os.chmod(path, stat.S_IWRITE)
    func(path)

def exponential_backoff(num_retries):
    base = 1  # Base wait time in seconds
    for n in range(num_retries):
        yield base * 2 ** n + random.uniform(0, 1)  # Add randomness to avoid simultaneous retries

repo_url = "https://github.com/entbappy/End-to-end-ML-Project-Implementation"
repo_path = "test_repo/"
if os.path.exists(repo_path):
    shutil.rmtree(repo_path, onerror=remove_readonly)
Repo.clone_from(repo_url, to_path=repo_path)
print(f"Repository cloned to {repo_path}")

loader = GenericLoader.from_filesystem(repo_path+'/src/mlProject', glob="**/*", suffixes=[".py"],
                                       parser=LanguageParser(language=Language.PYTHON, parser_threshold=500))
documents = loader.load()
document_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON, chunk_size=200, chunk_overlap=200)
text = document_splitter.split_documents(documents=documents)
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables.")
embeddings = OpenAIEmbeddings(disallowed_special=())

max_retries = 5
for wait_time in exponential_backoff(max_retries):
    try:
        vectordb = Chroma.from_documents(text, embedding=embeddings, persist_directory='./data')
        vectordb.persist()
        break
    except openai.Error as e:
        if e.__class__.__name__ == 'RateLimitError':
            print(f"Rate limit exceeded, retrying in {wait_time:.2f} seconds...")
            time.sleep(wait_time)
        else:
            raise  # Re-raise other errors not related to rate limits


Repository cloned to test_repo/


AttributeError: module 'openai' has no attribute 'Error'

In [ ]:
import os
import time
import random
import shutil
import stat
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
import openai

def remove_readonly(func, path, _):
    """Clear the readonly bit and reattempt the removal."""
    os.chmod(path, stat.S_IWRITE)
    func(path)

def exponential_backoff(num_retries):
    """Generator for exponential backoff times."""
    base = 1  # base wait time in seconds
    for n in range(num_retries):
        yield base * 2 ** n + random.uniform(0, 1)  # Add some randomness

# Define the repository URL and the path to clone to
repo_url = "https://github.com/entbappy/End-to-end-ML-Project-Implementation"
repo_path = "test_repo/"

# Check if the directory exists
if os.path.exists(repo_path):
    # If it exists, remove the directory with error handling
    shutil.rmtree(repo_path, onerror=remove_readonly)

# Clone the repository
Repo.clone_from(repo_url, to_path=repo_path)
print(f"Repository cloned to {repo_path}")

# Set up document loading and splitting
loader = GenericLoader.from_filesystem(repo_path+'/src/mlProject',
                                       glob="**/*",
                                       suffixes=[".py"],
                                       parser=LanguageParser(language=Language.PYTHON, parser_threshold=500))
documents = loader.load()
document_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON, chunk_size=200, chunk_overlap=200)
text = document_splitter.split_documents(documents=documents)

# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables.")

# Setup embeddings and vector database
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY, disallowed_special=())

# Implement retry logic for rate limiting
max_retries = 5
for wait_time in exponential_backoff(max_retries):
    try:
        vectordb = Chroma.from_documents(text, embedding=embeddings, persist_directory='./data')
        vectordb.persist()
        break
    except openai.APIError as e:
        if e.code == 429:  # Typically, HTTP 429 is the status code for Too Many Requests
            print(f"Rate limit exceeded, retrying in {wait_time:.2f} seconds...")
            time.sleep(wait_time)
        else:
            raise  # Re-raise the exception if it's not a rate limit error
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        break


Repository cloned to test_repo/


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
import openai

try:
    vectordb = Chroma.from_documents(text, embedding=embeddings, persist_directory='./data')
    vectordb.persist()
    break
except openai.APIError as e:
    if e.code == 429:  # Typically, HTTP 429 is the status code for Too Many Requests
        print(f"Rate limit exceeded, retrying in {wait_time:.2f} seconds...")
        time.sleep(wait_time)
    else:
        raise  # Rethrow the exception if it's not a rate limit error
except Exception as e:
    print(f"An unexpected error occurred: {str(e)}")
    break
